## Moduel

In [1]:
from transformers import (AdamW, 
                                               AlbertConfig, AlbertModel, AlbertTokenizer,
                                               BertConfig, BertModel, BertTokenizer,
                                               ElectraConfig, ElectraModel, ElectraTokenizer, ElectraForSequenceClassification,
                                               RobertaConfig, RobertaModel, RobertaTokenizer, RobertaForSequenceClassification,
                                               get_cosine_schedule_with_warmup,
                                               get_linear_schedule_with_warmup)
import nlp
import logging
from transformers import BertTokenizer, EncoderDecoderModel, Trainer, TrainingArguments
import torch
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM
import csv

In [2]:
import math
import os
import gc
import random
import time
from collections import Counter, defaultdict
from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm


In [19]:
dataset = pd.read_csv('dataset_csv.csv', names = ('id', 'highlights', 'article'))

In [20]:
dataset.head(3)

,id,highlights,article
0,００００００４３,年末ジャンボ宝くじ第２９６回全国自治宝くじ当選番号,◇年末ジャンボ宝くじ第２９６回全国自治宝くじ（３１日・東京宝塚劇場）◇１等（６０００万円）３...
1,００００００４４,ソウル市民歓喜、統一に希望の灯−−南北朝鮮・非核化共同宣言,【ソウル３１日薄木秀夫】南北朝鮮が三十一日、朝鮮半島の非核化の決意を世界に宣言した。ソウル市...
2,００００００４５,出生率９．９に落ち込み人口１００人から子供１人生まれず−−人口動態年間推計,出生率（人口千人当たりの出生数）が一九九一年に遂に九・九まで落ち込み、十二年連続で史上最低記...


In [18]:
dataset[1]

0                    年末ジャンボ宝くじ第２９６回全国自治宝くじ当選番号
1                ソウル市民歓喜、統一に希望の灯−−南北朝鮮・非核化共同宣言
2        出生率９．９に落ち込み人口１００人から子供１人生まれず−−人口動態年間推計
3                        内外とも多忙な年に天皇ご一家、健やかに新春
4                  遭難の４人依然、不明−−伊豆諸島・八丈島沖のヨット事故
                         ...                  
86023                 ［雑記帳］ＪＲ大津駅構内の壁に「比良の天狗参上」
86024                   田中貞美氏死去＝エスペランチスト護憲の会会長
86025                       障害男性、年金減額恐れ自殺？−−大阪
86026           大阪・福島区でアパート全焼、２人死亡独居高齢者入居、逃げ遅れ
86027                          兵庫の農協でコメ４・２トン盗難
Name: 1, Length: 86028, dtype: object

In [14]:
df[2][1]

5

## Config

In [4]:
config = {
    # True にすると訓練データの最初の1000個だけ使う
    'DEBUG': False,
    # 乱数のシードを固定する値
    'seed': 1234,
    # pretrained list: https://huggingface.co/transformers/pretrained_models.html
    'model_name': 'bert-base-uncased',
    'model': EncoderDecoderModel,
    'model_config': BertConfig,
    'tokenizer': BertJapaneseTokenizer,
    'do_lower_case': True,
    # Bert 内部のパラメータに対する学習率
    'encoder_lr': 1e-5,
    # Bert 以外のパラメータに対する学習率
    'decoder_lr': 1e-5,
    # 学習エポック数
    'epochs': 8,
    # batch_size x accum_steps サンプルごとにパラメータ更新
    'accum_steps': 1,
    # ミニバッチサイズ
    'batch_size': 16,
    # 学習率を動的に変える仕組みの選択とそのパラメータ
    'scheduler': 'linear', # ['linear', 'cosine', 'None']
    'num_cycles': 0.5, # cosine(float)
    'rate_warmup_steps': 0.15, # linear(int)
    # 入力文の最大トークン数．これ以上は **切り捨てられる**
    'encoder_max_length': 252,
    'decoder_max_length': 64,
    # 学習結果を保存するディレクトリ名
    'output_dir': './src/output/',
    # 学習結果を保存するファイル名
    'save_model_name': 'bert_summarization.bin',
    # ログファイルの prefix
    'save_log_name': 'log',
    # 入力データが存在する親ディレクトリ名
    'input_dir': './input/',
    # 入力ファイルが存在するディレクトリ名
    # -> ファイルは input_dir + input_fname + {train_all.jsonl, dev.jsonl, test.jsonl}
    # dataset from https://leaderboard.allenai.org/winogrande/submissions/get-started
    'input_fname': 'winogrande_1.0/',
    # mode = 'score' or 'loss' : 開発データ上で score (分類精度）かロスか，いずれが最良のものを保存するか指定
    'mode': 'score',
    # 搭載GPUを全部並列に使う（2020-09-08: matuzaki: hinoki で４枚GPU使うと二倍くらい速い．kayaX は未確認）
    'parallel': True,
    # Fine-tuning 済みのパラメータファイルを指定する
    'fine_tuned_model': None # './src/output/bert_winogrande.bin'
}

config['verbose'] = 10 if config['DEBUG'] else 0


## GPUの設定

In [5]:
# GPU が使えるときは使う
config['device'] = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config['num_device'] = torch.cuda.device_count()

# 一度に各GPUで（元々の）batch_size ずつ並列処理
if config['parallel']:
    config['batch_size'] = config['batch_size'] * config['num_device']


## Utils

In [6]:
# 乱数シードの固定
def seed_everything(seed=42):
    """Function for consistency of experiment
    input
        seed: random seed
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
# ログ出力設定
def get_logger(filename='log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

In [8]:
# 変更点あり

# Pandas のデータフレームとしてデータを読み込み．
#
# 訓練データは jsonl フォーマット（各行が一つの dictionary）で
# ファイル名は以下のようにする
#    訓練　 : 'input_dir' + 'input_fname' + train_all.jsonl
#    開発　 : 'input_dir' + 'input_fname' + dev.jsonl
#    テスト : 'input_dir' + 'input_fname' + test.jsonl
def load_data(config):
        
    train = pd.read_json(config['input_dir']+config['input_fname']+'train_all.jsonl', orient='records', lines=True)
    valid = pd.read_json(config['input_dir']+config['input_fname']+'dev.jsonl', orient='records', lines=True)
    test =  pd.read_json(config['input_dir']+config['input_fname']+'test.jsonl', orient='records', lines=True)
    
    if config['DEBUG']:
        train = train.head(1000)

    return train, valid, test

In [9]:
#変更点あり

# array を2つ渡してその一致率として精度を計算する
def get_score(label, prediction):
    score = accuracy_score(label, prediction)
    return score

In [10]:
# 変更点あり？

# 最適化アルゴリズムのパラメータを設定
def set_optimizer_params(model, config):
    # Pre-trained モデル（Bert）のパラメータか自前の部分のものか判別するのに使用
    def is_backbone(n):
        return 'bert' in n

    # モデルのパラメータを名前とともにリストにする
    param_optimizer = list(model.named_parameters())

    # Weight decay を適用しないパラメータの名前のパターン
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    
    # パラメータの種類ごとに最適化アルゴリズムのパラメータ（学習率・Weight decay の強さ）を設定
    optimizer_parameters = [
        # Bert の中のパラメータ．Weight decay させるもの
        {"params": [p for n, p in param_optimizer if is_backbone(n) and not any(nd in n for nd in no_decay)],
        "lr": config['encoder_lr'], "weight_decay": 0.01},
        # Bert の中のパラメータ．Weight decay させないもの
        {"params": [p for n, p in param_optimizer if is_backbone(n) and any(nd in n for nd in no_decay)],
        "lr": config['encoder_lr'], "weight_decay": 0.0},
        # Bert 以外のパラメータ．Weight decay させるもの
        {"params": [p for n, p in param_optimizer if not is_backbone(n) and not any(nd in n for nd in no_decay)],
        "lr": config['decoder_lr'], "weight_decay": 0.01},
        # Bert 以外のパラメータ．Weight decay させないもの
        {"params": [p for n, p in param_optimizer if not is_backbone(n) and any(nd in n for nd in no_decay)],
        "lr": config['decoder_lr'], "weight_decay": 0.0,},
    ]

    return optimizer_parameters

## Datasets


In [ ]:
# 変更点あり

# Winogrande データの加工
# 
# * sentence = "John likes Mary because _ is nice."
# * option1 = "John"
# * option2 = "Mary"
#
# だったら，代名詞の位置（"_"）を先行詞候補で置き換えて
#
# * input1 -> "[CLS] John likes Mary because John [SEP] is nice. [SEP]"
# * input2 -> "[CLS] John likes Mary because Mary [SEP] is nice. [SEP]"
#
# という２つの入力を作成する.
#
# また，config["max_length"] を最大トークン数（特殊トークンも含む）で打ち切り，
# それより短ければパディング（ゼロうめ）する
#
def convert_line(sentence, option1, option2, config, tokenizer):

    # データバグを直す
    if '_Laura' in sentence:
        sentence = sentence.replace('_Laura', 'Laura')
    elif '_ Matthew' in sentence:
        sentence = sentence.replace('_ Matthew', 'Matthew')
    
    # アンダースコア "_" で表された代名詞の前後で文を分けて，先行詞候補ふたつ（option1, option2）をそれぞれ付加
    sentence_s, sentence_e = sentence.split('_')
    sentence_s_1 = ' '.join([sentence_s, option1])
    sentence_s_2 = ' '.join([sentence_s, option2])

    # [...置き換えた代名詞位置] + [SEP] + [代名詞位置より後] と [SEP] を挟んで結合し tokenize
    inputs1 = tokenizer.encode_plus(sentence_s_1, text_pair=sentence_e, return_tensors=None, add_special_tokens=True, 
                    max_length=config['max_length'], pad_to_max_length=True, truncation=True)

    # tokenizer からの出力（トークン番号のリスト，attension mask など）をすべて torch の tensor に変換して
    # GPU 使用ならば GPU に置く
    for k, v in inputs1.items():
        inputs1[k] = torch.tensor(v, dtype=torch.long).to(config['device'])

    # もう一つの選択肢をはめこんだほうも同様に tokenize
    inputs2 = tokenizer.encode_plus(sentence_s_2, text_pair=sentence_e, return_tensors=None, add_special_tokens=True, 
                    max_length=config['max_length'], pad_to_max_length=True, truncation=True)
    for k, v in inputs2.items():
        inputs2[k] = torch.tensor(v, dtype=torch.long).to(config['device'])
    
    return inputs1, inputs2

In [ ]:
# データを保持するクラス．DataLoader のためのインタフェースを提供
class MyDataset:
    # sentence, ..., answer: データの各フィールドおよび正解ラベルを縦に並べたリストをフィールドごとに与える
    # config: 設定の dictionary
    # tokenizer: 使用する transformer モデル（Bert ほか）用の tokenizer
    def __init__(self, sentence, option1, option2, answer, config, tokenizer):
        self.sentence = sentence
        self.option1 = option1
        self.option2 = option2
        self.label = answer
        self.config = config
        self.tokenizer = tokenizer
    
    # 全データ数を返す
    def __len__(self):
        return len(self.sentence)
    
    # item 番目のデータを返す
    def __getitem__(self, item):
        # トークナイズおよび必要な入力の加工
        inputs1, inputs2 = convert_line(
            self.sentence[item], 
            self.option1[item],
            self.option2[item],
            self.config,
            self.tokenizer
        )
    
        # 辞書の形でデータを返す．正解ラベルはゼロ始まりになるように -1 している
        return {
            'inputs1': inputs1,
            'inputs2': inputs2,
            'label': torch.tensor(self.label[item] - 1, dtype=torch.long)
        }


## Trainer

In [ ]:
# 1 epoch の学習
#
def trainer(model, data_loader, optimizer, criterion, scheduler, config):
    # 勾配をためるモードに切り替え
    model.train()

    losses = []
    preds = []
    targets = []

    if config['verbose']:
        lossf = None
        accf = None

    # 勾配の和をクリア
    optimizer.zero_grad()

    for idx, batch in enumerate(data_loader):
        # モデルからの出力を logit として取得
        pred = model(
            batch['inputs1'],
            batch['inputs2'],
        )

        # 正解と比較してロスを計算
        loss =  criterion(pred, batch['label'].to(config['device']))

        # バックプロパゲーション
        loss.backward()

        # 設定したミニバッチ数を処理したらパラメータ更新
        if (idx + 1) % config['accum_steps'] == 0:
            optimizer.step()
            optimizer.zero_grad()
        
        # スケジューラ（学習率の動的調整）を進める
        if scheduler is not None:
            scheduler.step()
        
        # 経過報告のためにロスと訓練データに対する予測精度を保存
        losses.append(loss.detach().cpu().item())
        preds.extend(pred.cpu().detach().numpy().argmax(1))
        targets.extend(batch["label"].detach().cpu().numpy())
        
        # デバッグモードの時はミニバッチ10個ごとに経過を出力
        if config['verbose']:
            if lossf:
                lossf = 0.98 * lossf + 0.02 * loss.item()
            else:
                lossf = loss.item()        
            
            if ((idx + 1) % config['verbose']) == 0:
                logger.info("{} Train Loss : {:.4f}".format(idx+1, lossf))
    
    return np.mean(losses), get_score(targets, preds)

## Evaluator

In [ ]:
# 開発データに対するロスと精度を計算
#
def evaluator(model, data_loader, criterion, config):
    # 勾配をためないモード（評価用）に切り替え
    model.eval()

    losses = []
    preds = []
    targets = []

    # 勾配計算はしない
    with torch.no_grad():
        # 開発データをミニバッチに分けてロスと分類精度を計算
        for idx, batch in enumerate(data_loader):
            pred = model(
                batch['inputs1'],
                batch['inputs2'],
            )

            loss = criterion(pred, batch['label'].to(config['device']))
            
            losses.append(loss.detach().cpu().item())
            preds.extend(pred.cpu().detach().numpy().argmax(1))
            targets.extend(batch["label"].detach().cpu().numpy())
    
    return np.mean(losses), get_score(targets, preds)

## Model

In [ ]:
# Pre-trained モデルの上にかぶせる個別タスク用のモデルクラス
#
# 1. ２つの文（２つの選択肢に対応）を Bert にそれぞれ入力する
#
# 2. それぞれの [CLS] トークンに対応する出力ベクトルを
#    (BertModel 内部で）全結合層＋tanh に通したものを得る
#    * 以下のコメントではこれを単に「[CLS] トークンに対応するベクトル」と呼ぶ
#    * ここで使われる全結合層は next sentence prediction で Pre-training されたもの
#
# 3. ２文の [CLS] トークンに対応するベクトルとパラメータベクトルの内積を取ったものを
#    softmax に入れて各選択肢が正しい確率とみなす
#
class CustomModel(nn.Module):
    def __init__(self, config, model_config):
        super(CustomModel, self).__init__()

        # Pre-trained モデルのパラメータを読み込み
        self.bert = config['model'].from_pretrained(config['model_name'], config=model_config)

        # [CLS] トークンに対する出力ベクトルに適用する Drop-out レイヤ
        # drop-out 確率は Pre-trained モデルの訓練と同一のものを使用（？）
        self.dropout = nn.Dropout(model_config.hidden_dropout_prob)

        # [CLS] トークンに対する出力ベクトルと内積を取るパラメータベクトル
        self.classifier = nn.Linear(model_config.hidden_size, 1)

        # config で fine_tuned_model が
        # 指定された   -> それを読み込む
        # 指定されない -> Bert の pre-trained モデルを読み込み付加するパラメータを初期化
        if config["fine_tuned_model"]:
            # 空の Bert モデルを作る
            self.bert = config['model'](config=model_config)

            # Fine-tuning 済みのパラメータを読み込む
            self.load_state_dict(torch.load(config["fine_tuned_model"]))
        else:
            # Pre-trained モデルのパラメータを読み込み
            self.bert = config['model'].from_pretrained(config['model_name'], config=model_config)

            # 分類のためのパラメータベクトルを正規分布でランダム初期化
            nn.init.normal_(self.classifier.weight, std=0.02)
    
    # 確率値が欲しい時は forward の引数で softmax = True とする
    # それ以外のときは softmax をとる前の値（選択肢ごとのスコア = logit）を返す
    def forward(self, input1, input2, softmax=False):
        # ２つの選択肢に対応した２つの入力をそれぞれ Bert に通して [CLS] トークンに
        # 対応したベクトルを取り出す
        _, h1 = self.bert(**input1)
        _, h2 = self.bert(**input2)

        # Drop-out を適用してからパラメータベクトルと内積をとる
        h1 = self.dropout(h1)
        y_pred1 = self.classifier(h1)
        
        # もう片方の文に対する出力についても同じ
        h2 = self.dropout(h2)
        y_pred2 = self.classifier(h2)
        
        # ミニバッチ内の各サンプルに対する y_pred1, y_pred2 を並べて
        # batch_size x 2 の tensor にする
        logits = torch.cat((y_pred1.reshape(-1, 1), y_pred2.reshape(-1, 1)), dim=1)

        if softmax:
            logits = F.softmax(logits, dim=1)

        return logits

## Main

In [ ]:
logger = get_logger(config['output_dir'] + config['save_log_name'])

def main():

    logger.info("Config")
    for k, v in config.items():
        logger.info(f'   {k}:{v}')

    # データをファイルから読み込み
    train, valid, test = load_data(config)
    logger.info("train: {}, valid:{}, test: {}".format(len(train), len(valid), len(test)))

    # トークナイザを読み込み
    tokenizer = config['tokenizer'].from_pretrained(config['model_name'], do_lower_case=config['do_lower_case'])

    # 実験結果が再現できるように乱数シードを固定
    seed_everything(config['seed'])

    # 訓練データの DataLoader を準備
    train_dataset = MyDataset(train['sentence'].values, train['option1'].values, train['option2'].values, train['answer'].values, config, tokenizer)
    # 単に２文から正しいものを選ぶ場合
    # train_dataset = MyDataset(train['sentence1'].values, train['sentence2'].values, train['answer'].values, config, tokenizer)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=config['batch_size'])

    # 開発データの DataLoader を準備
    valid_dataset = MyDataset(valid['sentence'].values, valid['option1'].values, valid['option2'].values, valid['answer'].values, config, tokenizer)
    # 単に２文から正しいものを選ぶ場合
    # valid_dataset = MyDataset(valid['sentence1'].values, valid['sentence2'].values, valid['answer'].values, config, tokenizer)
    valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=config['batch_size'])

    # Pre-trained モデルの設定を読み込み
    model_config = config['model_config'].from_pretrained(config['model_name'])

    # Pre-trained モデルを自分のモデルに読み込む
    model = CustomModel(config, model_config)

    # 複数 GPU を並列に使う場合
    if config['parallel']:
        model = torch.nn.DataParallel(model)

    # 勾配をクリアして GPU に送る
    model.zero_grad()
    model.to(config['device'])

    # 最適化アルゴリズム（AdamW）の設定
    optimizer_grouped_parameters = set_optimizer_params(model, config)
    optimizer = AdamW(optimizer_grouped_parameters, lr=config['encoder_lr'], eps=1e-6)

    # パラメータ更新回数
    num_train_steps = math.floor(len(train_dataset)  * config['epochs'] / config['batch_size'])

    # スケジューラーの設定（パラメータ更新につれて学習率を調節する仕組み）
    if config['scheduler'] == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=math.floor(config['rate_warmup_steps'] * num_train_steps), num_training_steps=num_train_steps
        )
    elif config['scheduler'] == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=math.floor(config['rate_warmup_steps'] * num_train_steps), num_training_steps=num_train_steps, num_cycles=config['num_cycles']
        )
    else:
        scheduler = None

    # ロスはモデルからの logit の出力を softmax に通した確率の負の対数尤度
    criterion = nn.CrossEntropyLoss()

    best_score = 0.
    best_loss = 100.

    for epoch in range(config['epochs']):
        start = time.time()

        # 1 epoch 分パラメータ更新
        train_loss, train_score = trainer(model, train_loader, optimizer, criterion, scheduler, config)

        # 開発データ上でのロスと分類精度を測る
        val_loss, val_score = evaluator(model, valid_loader, criterion, config)

        logger.info(
            "Epoch {}: Train Loss {:.4f}, Valid Loss {:.4f}, Train score {:.4f}, Valid score {:.4f}, elapsed {:.4f}s".format(
                epoch + 1, train_loss, val_loss, train_score, val_score, time.time() - start)
            )
        
        # 設定に従って，分類精度かロスかがこれまでで最良ならモデルを保存
        new_best = False
        if config['mode'] == 'score':
            if val_score > best_score:
                new_best = True
        elif config['mode'] == 'loss':
            if val_loss < best_loss:
                new_best = True

        if new_best:
            best_loss = val_loss
            best_score = val_score
            if config['parallel']:
                torch.save(model.module.state_dict(), config['output_dir'] + config['save_model_name'])
            else:
                torch.save(model.state_dict(), config['output_dir'] + config['save_model_name'])
    
    logger.info('End of experiment!')
    logger.info('best loss: {:.4f}, score: {:.4f}'.format(best_loss, best_score))
    logger.info('')

    # GPU のメモリを解放
    del model
    torch.cuda.empty_cache()
    gc.collect()

## TestDataの評価

In [ ]:
def run_test():

    logger.info("Config")
    for k, v in config.items():
        logger.info(f'   {k}:{v}')

    # データをファイルから読み込み
    _, _, test = load_data(config)
    logger.info("# test instances: {}".format(len(test)))

    # トークナイザを読み込み
    tokenizer = config['tokenizer'].from_pretrained(config['model_name'], do_lower_case=config['do_lower_case'])

    # 実験結果が再現できるように乱数シードを固定
    seed_everything(config['seed'])

    # テストデータの DataLoader を準備
    test_dataset = MyDataset(test['sentence'].values, test['option1'].values, test['option2'].values, test['answer'].values, config, tokenizer)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=config['batch_size'])

    # Pre-trained モデルの設定を読み込み
    model_config = config['model_config'].from_pretrained(config['model_name'])

    # Fine-tuning 済みのモデルを自分のモデルに読み込む
    model = CustomModel(config, model_config)

    # 複数 GPU を並列に使う場合
    if config['parallel']:
        model = torch.nn.DataParallel(model)

    # 勾配をクリアして GPU に送る
    model.zero_grad()
    model.to(config['device'])

    # ロスはモデルからの logit の出力を softmax に通した確率の負の対数尤度
    criterion = nn.CrossEntropyLoss()

    start = time.time()

    # テストデータ上でのロスと分類精度を測る
    test_loss, test_score = evaluator(model, test_loader, criterion, config)

    logger.info(
        "Test Loss {:.4f}, Test score {:.4f}, elapsed {:.4f}s".format(
                test_loss, test_score, time.time() - start)
            )
        
    logger.info('End of test!')
    logger.info('')

    # GPU のメモリを解放
    del model
    torch.cuda.empty_cache()
    gc.collect()


if __name__ == '__main__':
    if config["fine_tuned_model"]:
        run_test()
    else:
        main()